## Module Import 

In [ ]:
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
import os
import time as t
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.python.keras.layers import Dense
np.set_printoptions(precision=8)
seed=98

print(tf. __version__)

2.7.0


In [ ]:
# change the directory to data location
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/PINN')

Mounted at /content/drive


In [ ]:
# Read the file
file = 'major_chemical_marker.h5'
f = h5py.File(file, 'r')

In [ ]:
# Get the feature and the label
# The dataset have larger points HHR region [data= Augumentation]
dataset = np.array(f.get('major_chemical_marker'))
print(dataset.shape)

(51966851, 5)


In [ ]:
# Separate the feature and the label
X_old = dataset[:,:-1]
nh2 = X_old[:,1:2]
oh = X_old[:,2:3]
X = np.c_[nh2,oh]
Y = dataset[:,[-1]]
#print(X,Y.shape)

In [ ]:
#shuffle the dataset
X,Y = shuffle(X, Y, random_state=seed)
print(X, Y)

[[3.19554703e-03 7.90773076e-04]
 [1.46000320e-03 2.29734957e-04]
 [9.57325610e-05 8.72315094e-03]
 ...
 [1.23655904e-04 9.34661739e-03]
 [8.52273733e-05 5.31098386e-03]
 [1.39086868e-03 1.05819294e-04]] [[5.4436895e+09]
 [1.4833097e+09]
 [1.4732527e+09]
 ...
 [1.5068352e+09]
 [9.4112301e+08]
 [1.7772887e+09]]


In [ ]:
print(np.amin(Y))

14080775.0


In [ ]:
# create the normalization fit 
scale_X = MinMaxScaler()
scale_Y = MinMaxScaler()

# The fit function
print(scale_X.fit(X))
print(scale_Y.fit(Y))

MinMaxScaler()
MinMaxScaler()


In [ ]:
#test_train split 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(46770165, 2) (5196686, 2) (46770165, 1) (5196686, 1)


In [ ]:
# normalize the train data and label
X_train_n = scale_X.transform(X_train)
Y_train_n = scale_Y.transform(Y_train)
print(X_train_n)
print(Y_train_n)

[[0.14476164 0.37797722]
 [0.25651807 0.31460327]
 [0.11479945 0.265395  ]
 ...
 [0.67491496 0.06919738]
 [0.4547599  0.01520755]
 [0.003989   0.41863513]]
[[0.20670448]
 [0.09260704]
 [0.02678744]
 ...
 [0.3950092 ]
 [0.09517243]
 [0.01614404]]


In [ ]:
# Create the chunk 
X_chunk = X_train_n[0:5000000,:]

Y_chunk = Y_train_n[0:5000000,:]
print(X_chunk, Y_chunk)

[[0.14476164 0.37797722]
 [0.25651807 0.31460327]
 [0.11479945 0.265395  ]
 ...
 [0.61361676 0.1546674 ]
 [0.04310646 0.58747154]
 [0.41030064 0.00824165]] [[0.20670448]
 [0.09260704]
 [0.02678744]
 ...
 [0.37362018]
 [0.02688141]
 [0.08041646]]


In [ ]:
# Creaate the batch slices.
train_data = tf.data.Dataset.from_tensor_slices((X_chunk, Y_chunk))
train_data = train_data.batch(1024, drop_remainder=True)
print(train_data)
# test_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
# test_data = test_data.batch(1024, drop_remainder=True)

<BatchDataset shapes: ((1024, 2), (1024, 1)), types: (tf.float32, tf.float32)>


In [ ]:
# Get the pretrained model
# Read the file
file = 'nh2_oh.h5'
model = tf.keras.models.load_model(file)

# Get summary
model.summary()


Model: "Heat_release_rate"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 h_layer1 (Dense)            (None, 40)                120       
                                                                 
 h_layer2 (Dense)            (None, 24)                984       
                                                                 
 h_layer3 (Dense)            (None, 24)                600       
                                                                 
 output_layers (Dense)       (None, 1)                 25        
                                                                 
Total params: 1,729
Trainable params: 1,729
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# The Ki the is respective exponents of the species massfraction and temperature, which could yeild us a perfect reconstruction of the HRR
#k1, k2, k3, k4 = tf.Variable([1.0], dtype=tf.float32),tf.Variable([-0.5], dtype=tf.float32),tf.Variable([-0.75], dtype=tf.float32),tf.Variable([1.0], dtype=tf.float32)
k1,k2,k3 = tf.Variable(-1.0),tf.Variable(-0.75),tf.Variable(1.75)

In [ ]:
# The customized mean square error, taking the batch data at any instance
def customized_MSE(true, pred):
    MSE = tf.reduce_mean(tf.square(true-pred))
    return MSE
# The customized loss taking the non-linear equation as residual
def total_mse(MSE_NN, X, Y, k1, k2, k3):# the MSE_NN is the mean squared error of the neural network 
    residual = tf.square(tf.math.log(Y) - (k1*tf.math.log(X[:,0:1]) + k2* tf.math.log(X[:,1:2]) + tf.math.log(k3)))
    MSE_func = tf.reduce_mean(residual)
    total =  MSE_NN + MSE_func
    return total

In [ ]:
# Optimizer 
# optimizers = tf.keras.optimizers.SGD(learning_rate=0.03,momentum=0.8)
optimizers = tf.keras.optimizers.Adam(
    learning_rate=0.009, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

In [ ]:
#Training with customized optimization technic (Automatic-Gradient)
epochs = 100
# lr_rate = 0.07


for epoch in range(epochs):
  print("Starting a new epoch",epoch+1)
  for step, (X_batch, Y_batch) in enumerate(train_data):
    with tf.GradientTape(persistent=True) as tape:
      Y_pred = model(X_batch)
      current_loss = customized_MSE(Y_batch,Y_pred)
      new_loss = total_mse(current_loss, X_batch, Y_batch, k1, k2, k3)

    # The gradient check  
    gradient = tape.gradient(new_loss, [k1,k2,k3])
    grad = tape.gradient(current_loss, model.trainable_variables)

    #Assign new variables to the model using optimizer isntead of sub assign
    optimizers.apply_gradients(zip(grad, model.trainable_variables))
    optimizers.apply_gradients(zip(gradient, [k1,k2,k3]))

    #Update the Ki to reduce the residual
    # k1.assign_sub(gradient[0]*lr_rate)
    # k2.assign_sub(gradient[1]*lr_rate)
    # k3.assign_sub(gradient[2]*lr_rate)
  
  print("k1:{} k2:{} k3:{} ".format(k1.numpy(),k2.numpy(),k3.numpy()))
  print("current_Loss: {} new_loss: {}".format(current_loss.numpy(), new_loss.numpy()))  

Starting a new epoch 1
k1:0.9514361023902893 k2:0.4080950915813446 k3:1.3517593145370483 
current_Loss: 0.0021727452985942364 new_loss: 0.33113572001457214
Starting a new epoch 2
k1:0.9542925357818604 k2:0.4103706479072571 k3:1.3586788177490234 
current_Loss: 0.002077183686196804 new_loss: 0.3310304880142212
Starting a new epoch 3
k1:0.9545577764511108 k2:0.41046157479286194 k3:1.3588250875473022 
current_Loss: 0.002053947187960148 new_loss: 0.33099403977394104
Starting a new epoch 4
k1:0.9545586109161377 k2:0.41046181321144104 k3:1.358825922012329 
current_Loss: 0.0020506721921265125 new_loss: 0.3309907019138336
Starting a new epoch 5
k1:0.9545586705207825 k2:0.4104618430137634 k3:1.3588260412216187 
current_Loss: 0.0020399068016558886 new_loss: 0.33097994327545166
Starting a new epoch 6
k1:0.9545586705207825 k2:0.4104618430137634 k3:1.3588260412216187 
current_Loss: 0.002035211306065321 new_loss: 0.3309752345085144
Starting a new epoch 7
k1:0.9545586705207825 k2:0.4104618430137634 k3